In [1]:
import numpy as np

from HMM import *
from toolbox import *

In [2]:
# Import and separate datasets
ERROR_RATE = 10  # 10% or 20%
train_set, test_set = load_db(error_rate=ERROR_RATE)
X_train = [[token[0] for token in word] for word in train_set]
y_train = [[token[1] for token in word] for word in train_set]
X_test = [[token[0] for token in word] for word in test_set]
y_test = [[token[1] for token in word] for word in test_set]

# Get states and observations sets
states, observations = get_observations_states(X_train, y_train)
print("{} states :\n{}".format(len(states), states))
print("{} observations :\n{}".format(len(observations), observations))

# Example from dataset
print("\nSample example (observation, état) :\n{}".format(train_set[3]))

26 states :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
26 observations :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

Sample example (observation, état) :
[('a', 'a'), ('c', 'c'), ('v', 'c'), ('o', 'o'), ('u', 'u'), ('n', 'n'), ('t', 't')]


In [3]:
# Initialize and train HMM
hmm1 = HMM(states, observations)
hmm1.EM(X_train)

1st order HMM created with: 
 * 26 states
 * 26 observations
-- EXPECTATION
EM LOOP: n_iter= 0
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 1
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 2
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 3
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 4
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 5
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 6
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 7
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 8
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 9
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 10
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 11
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 12
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 13
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 14
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 15
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 16
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter= 17
-- MINIMIZATION
-- EXPECTATION
EM LOOP: n_iter=

In [4]:
y_test_pred = hmm1.predict(X_test)
display_correction_stats(X_test, y_test, y_test_pred, name="HMM1")

HMM1 score on test set
 * accuracy on full words : 2.53%
 * accuracy on letters    : 7.69%
   > typos corrected      : 56 (0.77%)
   > typos not corrected  : 689 (9.41%)
   > typos added          : 6068 (82.90%)

Dummy score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 89.82%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 745 (10.18%)
   > typos added          : 0 (0.00%)


In [5]:
states = ['Healthy', 'Fever', 'Z_end']

In [6]:
observations = ['normal', 'cold', 'dizzy']

In [7]:
hmm_test = HMM(states, observations)  
print(hmm_test.omega_Y)
print(hmm_test.omega_X)

1st order HMM created with: 
 * 3 states
 * 3 observations
['Fever', 'Healthy', 'Z_end']
['cold', 'dizzy', 'normal']


In [8]:
hmm_test.transition_logproba = np.array([[0.59, 0.4, 0.01], [0.3, 0.69, 0.01], [0., 0., 1.]])

In [9]:
hmm_test.observation_logproba = np.array([[0.3, 0.6, 0.1], [0.4, 0.1, 0.5], [0.33, 0.33, 0.33]])

In [10]:
hmm_test.initial_state_logproba = np.array([0.4, 0.6, 0.0])

In [11]:
alpha = hmm_test.forward(observations_sequence=['normal', 'cold', 'dizzy'], decode=True)

In [12]:
beta = hmm_test.backward(observations_sequence=['normal', 'cold', 'dizzy'], decode=True)

In [13]:
print('alpha', alpha)
print('beta', beta)

alpha [[0.04       0.03408    0.02812032]
 [0.3        0.0892     0.007518  ]
 [0.         0.001122   0.00077708]]
beta [[0.0035016  0.0384     0.2       ]
 [0.021894   0.0412     0.03333333]
 [0.01249578 0.03707    0.11      ]]


In [14]:
def fwd_bkw(observations, states, start_prob, trans_prob, emm_prob, end_st):
    # forward part of the algorithm
    fwd = []
    f_prev = {}
    for i, observation_i in enumerate(observations):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = start_prob[st]
            else:
                prev_f_sum = sum(f_prev[k]*trans_prob[k][st] for k in states)

            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum

        fwd.append(f_curr)
        f_prev = f_curr

    p_fwd = sum(f_curr[k] * trans_prob[k][end_st] for k in states)
    print(fwd)

    # backward part of the algorithm
    bkw = []
    b_prev = {}
    for i, observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = trans_prob[st][end_st]
            else:
                b_curr[st] = sum(trans_prob[st][l] * emm_prob[l][observation_i_plus] * b_prev[l] for l in states)

        bkw.insert(0,b_curr)
        b_prev = b_curr
    print(bkw)
    p_bkw = sum(start_prob[l] * emm_prob[l][observations[0]] * b_curr[l] for l in states)

    # merging the two parts
    posterior = []
    for i in range(len(observations)):
        posterior.append({st: fwd[i][st] * bkw[i][st] / p_fwd for st in states})

    assert p_fwd == p_bkw
    return fwd, bkw, posterior


In [15]:
states = ('Healthy', 'Fever')
end_state = 'E'
 
observations = ('normal', 'cold', 'dizzy')
 
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
 
transition_probability = {
   'Healthy' : {'Healthy': 0.69, 'Fever': 0.3, 'E': 0.01},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.59, 'E': 0.01},
   }
 
emission_probability = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
   }
fwd_bkw(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability,
                   end_state)

[{'Healthy': 0.3, 'Fever': 0.04000000000000001}, {'Healthy': 0.0892, 'Fever': 0.03408}, {'Healthy': 0.007518, 'Fever': 0.028120319999999997}]
[{'Healthy': 0.0010418399999999998, 'Fever': 0.00109578}, {'Healthy': 0.00249, 'Fever': 0.00394}, {'Healthy': 0.01, 'Fever': 0.01}]


([{'Fever': 0.04000000000000001, 'Healthy': 0.3},
  {'Fever': 0.03408, 'Healthy': 0.0892},
  {'Fever': 0.028120319999999997, 'Healthy': 0.007518}],
 [{'Fever': 0.00109578, 'Healthy': 0.0010418399999999998},
  {'Fever': 0.00394, 'Healthy': 0.00249},
  {'Fever': 0.01, 'Healthy': 0.01}],
 [{'Fever': 0.1229889624426741, 'Healthy': 0.8770110375573259},
  {'Fever': 0.3767719690490461, 'Healthy': 0.623228030950954},
  {'Fever': 0.7890472951586943, 'Healthy': 0.2109527048413057}])

In [16]:
print(alpha)
print(beta)

[[0.04       0.03408    0.02812032]
 [0.3        0.0892     0.007518  ]
 [0.         0.001122   0.00077708]]
[[0.0035016  0.0384     0.2       ]
 [0.021894   0.0412     0.03333333]
 [0.01249578 0.03707    0.11      ]]
